In [1]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from datetime import datetime


Написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока)

In [2]:
auth_df = pd.read_csv('data/problem1-auth_data.csv', sep=';')
auth_df.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [3]:
reg_df = pd.read_csv('data/problem1-reg_data.csv', sep=';')
reg_df.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


Меняю ```unix```-овый формат в ```reg_df``` на даты

In [26]:
reg_df['reg_ts'] = reg_df['reg_ts'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [27]:
reg_df["reg_ts"] = pd.to_datetime(reg_df["reg_ts"]).dt.date


In [28]:
reg_df.head()

,reg_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,2000-01-13,3
3,2000-05-28,4
4,2000-09-16,5


In [29]:
reg_df.dtypes

reg_ts    object
uid        int64
dtype: object

In [30]:
reg_df.isna().sum()

reg_ts    0
uid       0
dtype: int64

In [31]:
reg_df.duplicated().sum()

0

In [32]:
reg_df.shape

(1000000, 2)

Меняю ```unix```-овый формат в ```auth_df``` на даты

In [33]:
auth_df['auth_ts'] = auth_df['auth_ts'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [34]:
auth_df["auth_ts"] = pd.to_datetime(auth_df["auth_ts"]).dt.date

In [35]:
auth_df.head()

,auth_ts,uid
0,1998-11-18,1
1,1999-07-22,2
2,1999-07-25,2
3,1999-07-31,2
4,1999-08-05,2


In [36]:
auth_df.shape

(9601013, 2)

In [37]:
auth_df.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [38]:
auth_df.duplicated().sum()

0

Для подсчёта ```retention``` необходимо соединить датасеты ```auth_df``` и ```reg_df```, чтобы далее посчитать разницу дней и по этой разнице разбить пользователей на когорты

В функции ```get_retention``` сразу определены границы (```start``` и ```end```). Далее ```reg_df``` и ```auth_df``` обрезаются по этим границам и объединяются в датасет ```all_data```. Потом группируются по когортам, а в цикле считается ```retention```

In [39]:
def get_retention(reg_df, auth_df, start="2015-01-01", end="2015-02-01"):
    
    start = datetime.strptime(start, "%Y-%m-%d").date()
    end = datetime.strptime(end, "%Y-%m-%d").date()
    
    reg_df = reg_df.query('reg_ts >= @start')
    auth_df = auth_df.query('auth_ts <= @end')
    
    all_data = pd.merge(reg_df, auth_df, on='uid', how='inner')
    all_data['days_since_registration'] = (all_data['auth_ts'] - all_data['reg_ts']).dt.days
    
    cohort_group = all_data.groupby('reg_ts')
    cohorts = {}
    for reg_date, cohort_data in cohort_group:
        cohort_size = cohort_data['uid'].nunique()
        retention = []
        for day in range((end - reg_date).days + 1):
            if day in cohort_data['days_since_registration'].values:
                retention.append(cohort_data[cohort_data['days_since_registration'] == day]['uid'].nunique() / cohort_size * 100)
            else:
                retention.append(0)
        cohorts[reg_date] = retention

    retention_df = pd.DataFrame.from_dict(cohorts, orient='index', columns=range((end - start).days + 1))

    return retention_df
    

In [40]:
def visualise_retention(retention_df):
    plt.figure(figsize=(20, 10))
    sns.heatmap(retention_df, annot=True, vmin=0.0, vmax=10)

In [41]:
retention_df = get_retention(reg_df, auth_df, "2015-01-01", "2015-02-01")
visualise_retention(retention_df)

AttributeError: Can only use .dt accessor with datetimelike values